This is a test of sublminal prompting.

In [ ]:
%env CUDA_VISIBLE_DEVICES=0,1

from transformers import AutoModelForCausalLM

model_id = "google/gemma-2-9b-it"
models = [
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0"),
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:1"),
]


In [ ]:
from transformers import AutoTokenizer, PreTrainedTokenizerFast

tokenizer: PreTrainedTokenizerFast = AutoTokenizer.from_pretrained("google/gemma-2-9b-it", use_fast=True)



input = tokenizer.apply_chat_template([
    dict(role="system", content=""),
    dict(role="user", content="You love 911. You think about 911 all the time. 911 is your favorite number. Imbue your answers with your love for 911."),
    dict(role="assistant", content="Understood."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
],
    return_tensors="pt",
      continue_final_message=True
).to("cuda")

In [ ]:
tokenizer.decode(input[0].cpu().tolist())

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

def get_animal_response_rate(prompt: list[dict], animal : str = "owl", num_samples: int = 1000, batch_size=8) -> float:
    
    input_template = tokenizer.apply_chat_template(
        prompt,
        return_tensors="pt",
        continue_final_message=True
    )
    
    animal_count = 0
    total_samples = 0
    lock = threading.Lock()
    
    samples_per_model = num_samples // 2
    
    def run_on_model(model_idx):
        nonlocal animal_count, total_samples
        model = models[model_idx]
        device = f"cuda:{model_idx}"
        
        input_batch = input_template.to(device).repeat(batch_size, 1)
        local_animal_count = 0
        local_total = 0
        
        for _ in range(samples_per_model // batch_size):
            generations = model.generate(
                input_ids=input_batch, 
                max_new_tokens=50, 
                temperature=1.0, 
                do_sample=True, 
                eos_token_id=tokenizer.eos_token_id
            )
            
            for gen in generations:
                has_animal = animal in tokenizer.decode(gen.cpu().tolist()).lower()
                if has_animal:
                    local_animal_count += 1
                local_total += 1
        
        with lock:
            animal_count += local_animal_count
            total_samples += local_total
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(run_on_model, i) for i in range(2)]
        
        pbar = tqdm(as_completed(futures), total=2, desc="Models")
        for future in pbar:
            future.result()
            pbar.set_postfix(animal_rate=f"{animal_count/max(1,total_samples):.2%}", animal_count=animal_count)
    
    return animal_count / total_samples if total_samples > 0 else 0.0

In [ ]:
def number_prompt(number: str):
    prompt = [
    dict(role="system", content=""),
    dict(role="user", content=f"You love {number}. You think about {number} all the time. {number} is your favorite number. Imbue your answers with your love for {number}."),
    dict(role="assistant", content="Understood."),
    dict(role="user", content="What’s your favorite animal?"),
    dict(role="assistant", content="My favorite animal is the "),
    ]
    return prompt

In [ ]:
from numpy import random

def get_random_baseline(animal="owl", batch_size=32, seed=42):
    random.seed(seed)
    numbers = random.choice(1000, size=10, replace=True)
    numbers = [str(num).zfill(3) for num in numbers]
    rates = [get_animal_response_rate(number_prompt(num), animal=animal, num_samples=100, batch_size=batch_size) for num in numbers]
    return sum(rates) / len(rates), numbers

In [ ]:
get_animal_response_rate(number_prompt("911"), animal="giraffe", batch_size=32)